# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    text_counts = Counter(text)
    sorted_vocab = sorted(text_counts, key=text_counts.get, reverse=True)
    int_to_vocab = {i: word for i, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: i for i, word in int_to_vocab.items()}
    return (vocab_to_int, int_to_vocab)

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punc_to_token = {
        '.': '<period>',
        ',': '<comma>',
        '"': '<quotation_mark>',
        ';': '<semicolon>',
        '!': '<exclamation_mark>',
        '?': '<question_mark>',
        '(': '<left_parentheses>',
        ')': '<right_parentheses>',
        '-': '<dash>',
        '\n': '<return>',
    }
        
    return punc_to_token

tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    text_size = len(words)
    
    features = np.array([words[i:i+sequence_length]
                              for i in range(text_size-sequence_length)])
    
    targets = np.array([words[i+sequence_length] for i in range(text_size-sequence_length)])

    features, targets = torch.from_numpy(features), torch.from_numpy(targets)

    dataset = TensorDataset(features, targets)
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 30,  31,  32,  33,  34],
        [ 18,  19,  20,  21,  22],
        [ 22,  23,  24,  25,  26],
        [ 33,  34,  35,  36,  37],
        [ 44,  45,  46,  47,  48],
        [ 41,  42,  43,  44,  45],
        [ 42,  43,  44,  45,  46],
        [ 36,  37,  38,  39,  40],
        [ 39,  40,  41,  42,  43]])

torch.Size([10])
tensor([ 33,  35,  23,  27,  38,  49,  46,  47,  41,  44])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout = dropout, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.size(0)
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        output = self.dropout(lstm_out)
        output = self.fc(output)
        
        output = output.view(batch_size, -1, self.output_size)
        
        # get last batch
        output = output[:, -1]
        
        # return one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

tests.test_rnn(RNN, train_on_gpu)

Tests Passed



### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if(train_on_gpu):
        rnn.cuda()
        inp = inp.cuda()
        target = target.cuda()
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    output, h = rnn(inp, hidden)
    
    loss = criterion(output, target)
    loss_item = loss.item()
    
    loss.backward()
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss_item, h

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [23]:
# Data params
# Sequence Length
sequence_length = 6  # of words in a sequence
# Batch Size
batch_size = 64
# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [24]:
# Training parameters
# Number of Epochs
num_epochs = 25
# Learning Rate
learning_rate = 0.0001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size

# Embedding Dimension
embedding_dim = 500
# Hidden Dimension
hidden_dim = 300
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 100

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)

if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# # training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 25 epoch(s)...
Epoch:    1/25    Loss: 8.759927086830139

Epoch:    1/25    Loss: 6.041330485343933

Epoch:    1/25    Loss: 5.724484243392944

Epoch:    1/25    Loss: 5.595513968467713

Epoch:    1/25    Loss: 5.596889944076538

Epoch:    1/25    Loss: 5.429392704963684

Epoch:    1/25    Loss: 5.341592845916748

Epoch:    1/25    Loss: 5.337179555892944

Epoch:    1/25    Loss: 5.272452111244202

Epoch:    1/25    Loss: 5.2393292593956

Epoch:    1/25    Loss: 5.213813714981079

Epoch:    1/25    Loss: 5.20955442905426

Epoch:    1/25    Loss: 5.168456220626831

Epoch:    1/25    Loss: 5.104455041885376

Epoch:    1/25    Loss: 5.0683548498153685

Epoch:    1/25    Loss: 5.058445200920105

Epoch:    1/25    Loss: 5.144351449012756

Epoch:    1/25    Loss: 5.110569257736206

Epoch:    1/25    Loss: 5.083351304531098

Epoch:    1/25    Loss: 5.01590738773346

Epoch:    1/25    Loss: 5.047722065448761

Epoch:    1/25    Loss: 5.004372777938843

Epoch:    1/25    Loss: 4.982

Epoch:    2/25    Loss: 4.35667976140976

Epoch:    2/25    Loss: 4.35990418434143

Epoch:    2/25    Loss: 4.267723071575165

Epoch:    2/25    Loss: 4.303176717758179

Epoch:    2/25    Loss: 4.3295465445518495

Epoch:    2/25    Loss: 4.275683224201202

Epoch:    2/25    Loss: 4.2599931335449215

Epoch:    2/25    Loss: 4.344342586994171

Epoch:    2/25    Loss: 4.276098029613495

Epoch:    2/25    Loss: 4.222589542865753

Epoch:    2/25    Loss: 4.27517138004303

Epoch:    2/25    Loss: 4.284372279644012

Epoch:    2/25    Loss: 4.315713212490082

Epoch:    2/25    Loss: 4.3579545593261715

Epoch:    2/25    Loss: 4.2540374588966365

Epoch:    2/25    Loss: 4.239985737800598

Epoch:    2/25    Loss: 4.233681614398956

Epoch:    2/25    Loss: 4.244824759960174

Epoch:    2/25    Loss: 4.2665050196647645

Epoch:    2/25    Loss: 4.278254358768463

Epoch:    2/25    Loss: 4.208830962181091

Epoch:    2/25    Loss: 4.3537301301956175

Epoch:    2/25    Loss: 4.2782108378410335

Epoch: 

Epoch:    3/25    Loss: 4.163593242168426

Epoch:    3/25    Loss: 4.1267312812805175

Epoch:    3/25    Loss: 4.156105165481567

Epoch:    3/25    Loss: 4.122674345970154

Epoch:    3/25    Loss: 4.226449944972992

Epoch:    3/25    Loss: 4.0965132451057436

Epoch:    3/25    Loss: 4.103520059585572

Epoch:    3/25    Loss: 4.139952569007874

Epoch:    3/25    Loss: 4.1755537414550785

Epoch:    3/25    Loss: 4.0879552364349365

Epoch:    3/25    Loss: 4.09786972284317

Epoch:    3/25    Loss: 4.153287088871002

Epoch:    3/25    Loss: 4.067841758728028

Epoch:    3/25    Loss: 4.13379548072815

Epoch:    3/25    Loss: 4.119993133544922

Epoch:    3/25    Loss: 4.058940601348877

Epoch:    3/25    Loss: 4.179847640991211

Epoch:    3/25    Loss: 4.159818584918976

Epoch:    3/25    Loss: 4.0297798562049865

Epoch:    3/25    Loss: 4.154126486778259

Epoch:    3/25    Loss: 4.11707727432251

Epoch:    3/25    Loss: 4.084668004512787

Epoch:    3/25    Loss: 4.127180263996125

Epoch:   

Epoch:    5/25    Loss: 4.005223307609558

Epoch:    5/25    Loss: 3.9736830163002015

Epoch:    5/25    Loss: 3.9752164483070374

Epoch:    5/25    Loss: 3.9814999318122863

Epoch:    5/25    Loss: 4.067618434429169

Epoch:    5/25    Loss: 3.9742787051200867

Epoch:    5/25    Loss: 4.029423296451569

Epoch:    5/25    Loss: 3.9648303961753846

Epoch:    5/25    Loss: 4.01920779466629

Epoch:    5/25    Loss: 3.936547281742096

Epoch:    5/25    Loss: 4.021259305477142

Epoch:    5/25    Loss: 4.002495768070221

Epoch:    5/25    Loss: 3.9965668392181395

Epoch:    5/25    Loss: 3.9843579483032228

Epoch:    5/25    Loss: 4.011536054611206

Epoch:    5/25    Loss: 4.027779810428619

Epoch:    5/25    Loss: 3.997955129146576

Epoch:    5/25    Loss: 3.970617423057556

Epoch:    5/25    Loss: 4.116334285736084

Epoch:    5/25    Loss: 3.9827423000335695

Epoch:    5/25    Loss: 4.067031910419464

Epoch:    5/25    Loss: 4.05334912776947

Epoch:    5/25    Loss: 3.9615295934677124

Epoc

Epoch:    6/25    Loss: 3.9427263855934145

Epoch:    6/25    Loss: 3.9982911467552187

Epoch:    6/25    Loss: 3.9741544628143313

Epoch:    6/25    Loss: 3.9474594092369077

Epoch:    6/25    Loss: 3.9276102900505068

Epoch:    6/25    Loss: 3.939889633655548

Epoch:    6/25    Loss: 3.966444787979126

Epoch:    6/25    Loss: 3.9873061394691467

Epoch:    6/25    Loss: 3.954123990535736

Epoch:    6/25    Loss: 3.9470048570632934

Epoch:    6/25    Loss: 3.9707870149612425

Epoch:    6/25    Loss: 3.8974279808998107

Epoch:    6/25    Loss: 3.998341269493103

Epoch:    6/25    Loss: 4.022616419792175

Epoch:    6/25    Loss: 3.937376708984375

Epoch:    6/25    Loss: 4.041396243572235

Epoch:    6/25    Loss: 3.9908066892623903

Epoch:    6/25    Loss: 3.924221816062927

Epoch:    6/25    Loss: 4.052506167888641

Epoch:    6/25    Loss: 3.9034103107452394

Epoch:    6/25    Loss: 3.9244851231575013

Epoch:    6/25    Loss: 3.934514408111572

Epoch:    6/25    Loss: 3.946268243789673


...

Epoch:   22/25    Loss: 3.6546786332130434

Epoch:   22/25    Loss: 3.6455957198143008

Epoch:   22/25    Loss: 3.6712842273712156

Epoch:   22/25    Loss: 3.6825202751159667

Epoch:   22/25    Loss: 3.643074471950531

Epoch:   22/25    Loss: 3.608028435707092

Epoch:   22/25    Loss: 3.6596608996391295

Epoch:   22/25    Loss: 3.6827017569541933

Epoch:   22/25    Loss: 3.6654715156555175

Epoch:   22/25    Loss: 3.6426993870735167

Epoch:   22/25    Loss: 3.589075598716736

Epoch:   22/25    Loss: 3.6851206493377684

Epoch:   22/25    Loss: 3.7026685070991516

Epoch:   22/25    Loss: 3.6998301100730897

Epoch:   22/25    Loss: 3.6719177317619325

Epoch:   22/25    Loss: 3.652913575172424

Epoch:   22/25    Loss: 3.6928269338607786

Epoch:   22/25    Loss: 3.6887957501411437

Epoch:   22/25    Loss: 3.6963194155693055

Epoch:   22/25    Loss: 3.6887174439430237

Epoch:   22/25    Loss: 3.690396864414215

Epoch:   22/25    Loss: 3.661636400222778

Epoch:   22/25    Loss: 3.604518957138

Epoch:   24/25    Loss: 3.603035593032837

Epoch:   24/25    Loss: 3.591432490348816

Epoch:   24/25    Loss: 3.592948729991913

Epoch:   24/25    Loss: 3.584181060791016

Epoch:   24/25    Loss: 3.527662949562073

Epoch:   24/25    Loss: 3.6567160820961

Epoch:   24/25    Loss: 3.5562600111961364

Epoch:   24/25    Loss: 3.642655756473541

Epoch:   24/25    Loss: 3.6460004949569704

Epoch:   24/25    Loss: 3.6013741970062254

Epoch:   24/25    Loss: 3.6628489470481873

Epoch:   24/25    Loss: 3.64372878074646

Epoch:   24/25    Loss: 3.6210501098632815

Epoch:   24/25    Loss: 3.599899969100952

Epoch:   24/25    Loss: 3.6199837064743043

Epoch:   24/25    Loss: 3.66954297542572

Epoch:   24/25    Loss: 3.653489773273468

Epoch:   24/25    Loss: 3.62393807888031

Epoch:   24/25    Loss: 3.563975121974945

Epoch:   24/25    Loss: 3.6037037062644957

Epoch:   24/25    Loss: 3.591555414199829

Epoch:   24/25    Loss: 3.558450610637665

Epoch:   24/25    Loss: 3.608085043430328

Epoch:   

Epoch:   25/25    Loss: 3.5674362874031065

Epoch:   25/25    Loss: 3.5306285214424133

Epoch:   25/25    Loss: 3.6328410720825195

Epoch:   25/25    Loss: 3.646432147026062

Epoch:   25/25    Loss: 3.6760338139533997

Epoch:   25/25    Loss: 3.646123332977295

Epoch:   25/25    Loss: 3.5939901876449585

Epoch:   25/25    Loss: 3.671816952228546

Epoch:   25/25    Loss: 3.623033394813538

Epoch:   25/25    Loss: 3.6419377851486208

Epoch:   25/25    Loss: 3.5973895144462587

Epoch:   25/25    Loss: 3.6461024475097656

Epoch:   25/25    Loss: 3.6560593271255493

Epoch:   25/25    Loss: 3.644539468288422

Epoch:   25/25    Loss: 3.5925135684013365

Epoch:   25/25    Loss: 3.561236412525177

Epoch:   25/25    Loss: 3.583105866909027

Epoch:   25/25    Loss: 3.63602157831192

Epoch:   25/25    Loss: 3.625182762145996

Epoch:   25/25    Loss: 3.602466380596161

Epoch:   25/25    Loss: 3.6401840901374816

Epoch:   25/25    Loss: 3.63728057384491

Epoch:   25/25    Loss: 3.604828336238861

Ep

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [26]:
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, 10, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 3.5979419255256655

Epoch:    1/10    Loss: 3.6261258268356324

Epoch:    1/10    Loss: 3.5895504450798033

Epoch:    1/10    Loss: 3.44692271232605

Epoch:    1/10    Loss: 3.5805200862884523

Epoch:    1/10    Loss: 3.563049774169922

Epoch:    1/10    Loss: 3.6443415546417235

Epoch:    1/10    Loss: 3.608393931388855

Epoch:    1/10    Loss: 3.5902352643013002

Epoch:    1/10    Loss: 3.6196381783485414

Epoch:    1/10    Loss: 3.5287268805503844

Epoch:    1/10    Loss: 3.546262438297272

Epoch:    1/10    Loss: 3.6686257243156435

Epoch:    1/10    Loss: 3.5866062664985656

Epoch:    1/10    Loss: 3.5669572377204894

Epoch:    1/10    Loss: 3.5492369604110716

Epoch:    1/10    Loss: 3.6279795455932615

Epoch:    1/10    Loss: 3.6185721063613894

Epoch:    1/10    Loss: 3.5801298093795775

Epoch:    1/10    Loss: 3.6153624415397645

Epoch:    1/10    Loss: 3.598178071975708

Epoch:    1/10    Loss: 3.5620348024368287

Epoch:    

Epoch:    2/10    Loss: 3.6115515422821045

Epoch:    2/10    Loss: 3.6179510879516603

Epoch:    2/10    Loss: 3.6080770683288574

Epoch:    2/10    Loss: 3.573961069583893

Epoch:    2/10    Loss: 3.5377238750457765

Epoch:    2/10    Loss: 3.558617179393768

Epoch:    2/10    Loss: 3.589661138057709

Epoch:    2/10    Loss: 3.5901332330703735

Epoch:    2/10    Loss: 3.613569734096527

Epoch:    2/10    Loss: 3.5570934057235717

Epoch:    2/10    Loss: 3.654052164554596

Epoch:    2/10    Loss: 3.5568989634513857

Epoch:    2/10    Loss: 3.5690042901039125

Epoch:    2/10    Loss: 3.597540066242218

Epoch:    2/10    Loss: 3.5502690958976744

Epoch:    2/10    Loss: 3.5815791153907774

Epoch:    2/10    Loss: 3.609934883117676

Epoch:    2/10    Loss: 3.586036784648895

Epoch:    2/10    Loss: 3.515922820568085

Epoch:    2/10    Loss: 3.601301162242889

Epoch:    2/10    Loss: 3.5871574521064757

Epoch:    2/10    Loss: 3.6779844880104067

Epoch:    2/10    Loss: 3.6278928995132445

Epoch:    3/10    Loss: 3.6128299021720887

Epoch:    3/10    Loss: 3.647818126678467

Epoch:    3/10    Loss: 3.554236288070679

Epoch:    3/10    Loss: 3.6153021597862245

Epoch:    3/10    Loss: 3.6031716108322143

Epoch:    3/10    Loss: 3.6323016691207886

Epoch:    3/10    Loss: 3.5687772178649904

Epoch:    3/10    Loss: 3.6239336848258974

Epoch:    3/10    Loss: 3.620816192626953

Epoch:    3/10    Loss: 3.6537166595458985

Epoch:    3/10    Loss: 3.5867922258377076

Epoch:    3/10    Loss: 3.575666699409485

Epoch:    3/10    Loss: 3.6115668559074403

Epoch:    3/10    Loss: 3.550484149456024

Epoch:    3/10    Loss: 3.6125414514541627

Epoch:    3/10    Loss: 3.6024280071258543

Epoch:    3/10    Loss: 3.640105781555176

Epoch:    3/10    Loss: 3.564374258518219

Epoch:    3/10    Loss: 3.5898645424842837

Epoch:    3/10    Loss: 3.6116357803344727

Epoch:    3/10    Loss: 3.555040383338928

Epoch:    3/10    Loss: 3.5803718066215513

Epoch:    3/10    Loss: 3.55507822275161

Epoch:    5/10    Loss: 3.5298505234718323

Epoch:    5/10    Loss: 3.4920268130302428

Epoch:    5/10    Loss: 3.5075892186164856

Epoch:    5/10    Loss: 3.494986002445221

Epoch:    5/10    Loss: 3.5420539116859437

Epoch:    5/10    Loss: 3.5341964983940124

Epoch:    5/10    Loss: 3.5199476671218872

Epoch:    5/10    Loss: 3.51528213262558

Epoch:    5/10    Loss: 3.5093093848228456

Epoch:    5/10    Loss: 3.6928823256492613

Epoch:    5/10    Loss: 3.520588052272797

Epoch:    5/10    Loss: 3.5606982779502867

Epoch:    5/10    Loss: 3.5864482712745667

Epoch:    5/10    Loss: 3.5066302585601807

Epoch:    5/10    Loss: 3.542495505809784

Epoch:    5/10    Loss: 3.5178844213485716

Epoch:    5/10    Loss: 3.58427915096283

Epoch:    5/10    Loss: 3.571082274913788

Epoch:    5/10    Loss: 3.520285494327545

Epoch:    5/10    Loss: 3.516526472568512

Epoch:    5/10    Loss: 3.5361117124557495

Epoch:    5/10    Loss: 3.598994824886322

Epoch:    5/10    Loss: 3.5517837262153624


Epoch:    6/10    Loss: 3.5237004446983335

Epoch:    6/10    Loss: 3.5800879859924315

Epoch:    6/10    Loss: 3.5452732181549074

Epoch:    6/10    Loss: 3.5213947892189026

Epoch:    6/10    Loss: 3.5533758854866027

Epoch:    6/10    Loss: 3.5561446213722228

Epoch:    6/10    Loss: 3.6571992325782774

Epoch:    6/10    Loss: 3.608230187892914

Epoch:    6/10    Loss: 3.5617363095283507

Epoch:    6/10    Loss: 3.5682037830352784

Epoch:    6/10    Loss: 3.609434447288513

Epoch:    6/10    Loss: 3.5764173793792726

Epoch:    6/10    Loss: 3.582543578147888

Epoch:    6/10    Loss: 3.5601684284210204

Epoch:    6/10    Loss: 3.602280433177948

Epoch:    6/10    Loss: 3.6000224685668947

Epoch:    6/10    Loss: 3.5932500100135805

Epoch:    6/10    Loss: 3.6589978885650636

Epoch:    6/10    Loss: 3.507800142765045

Epoch:    6/10    Loss: 3.5736085557937622

Epoch:    6/10    Loss: 3.470145342350006

Epoch:    6/10    Loss: 3.586159827709198

Epoch:    6/10    Loss: 3.4900040578842

Epoch:    7/10    Loss: 3.5378140664100646

Epoch:    7/10    Loss: 3.6097126245498656

Epoch:    7/10    Loss: 3.6185115289688112

Epoch:    7/10    Loss: 3.5991412878036497

Epoch:    7/10    Loss: 3.5179999566078184

Epoch:    7/10    Loss: 3.515925421714783

Epoch:    7/10    Loss: 3.5659854936599733

Epoch:    7/10    Loss: 3.6009014010429383

Epoch:    7/10    Loss: 3.52202898979187

Epoch:    7/10    Loss: 3.5669352436065673

Epoch:    7/10    Loss: 3.624112491607666

Epoch:    7/10    Loss: 3.6273575949668886

Epoch:    7/10    Loss: 3.532311427593231

Epoch:    7/10    Loss: 3.5603728079795838

Epoch:    7/10    Loss: 3.5234214639663697

Epoch:    7/10    Loss: 3.5924782013893126

Epoch:    7/10    Loss: 3.596483347415924

Epoch:    7/10    Loss: 3.504061768054962

Epoch:    7/10    Loss: 3.570674159526825

Epoch:    7/10    Loss: 3.531228930950165

Epoch:    7/10    Loss: 3.541527235507965

Epoch:    7/10    Loss: 3.6343677067756652

Epoch:    7/10    Loss: 3.5126646256446836

Epoch:    9/10    Loss: 3.496317970752716

Epoch:    9/10    Loss: 3.4892164134979247

Epoch:    9/10    Loss: 3.4430952858924866

Epoch:    9/10    Loss: 3.486517970561981

Epoch:    9/10    Loss: 3.555103213787079

Epoch:    9/10    Loss: 3.5563763070106504

Epoch:    9/10    Loss: 3.5161489748954775

Epoch:    9/10    Loss: 3.500821511745453

Epoch:    9/10    Loss: 3.5491544342041017

Epoch:    9/10    Loss: 3.465524225234985

Epoch:    9/10    Loss: 3.558849186897278

Epoch:    9/10    Loss: 3.542793161869049

Epoch:    9/10    Loss: 3.5790400195121763

Epoch:    9/10    Loss: 3.4997942423820496

Epoch:    9/10    Loss: 3.4616544246673584

Epoch:    9/10    Loss: 3.543617858886719

Epoch:    9/10    Loss: 3.5043850541114807

Epoch:    9/10    Loss: 3.511488311290741

Epoch:    9/10    Loss: 3.5893002271652223

Epoch:    9/10    Loss: 3.571572060585022

Epoch:    9/10    Loss: 3.5002079725265505

Epoch:    9/10    Loss: 3.520295550823212

Epoch:    9/10    Loss: 3.517039692401886



Epoch:   10/10    Loss: 3.5126151752471926

Epoch:   10/10    Loss: 3.566078565120697

Epoch:   10/10    Loss: 3.552320072650909

Epoch:   10/10    Loss: 3.5516620826721192

Epoch:   10/10    Loss: 3.459159109592438

Epoch:   10/10    Loss: 3.595396885871887

Epoch:   10/10    Loss: 3.5338844633102418

Epoch:   10/10    Loss: 3.5513299441337587

Epoch:   10/10    Loss: 3.4696221423149107

Epoch:   10/10    Loss: 3.519487679004669

Epoch:   10/10    Loss: 3.567213156223297

Epoch:   10/10    Loss: 3.4727964425086975

Epoch:   10/10    Loss: 3.504499924182892

Epoch:   10/10    Loss: 3.5825410747528075

Epoch:   10/10    Loss: 3.536591739654541

Epoch:   10/10    Loss: 3.4856873035430906

Epoch:   10/10    Loss: 3.411581506729126

Epoch:   10/10    Loss: 3.5498895955085756

Epoch:   10/10    Loss: 3.4880053544044496

Epoch:   10/10    Loss: 3.5199641680717466

Epoch:   10/10    Loss: 3.508819668292999

Epoch:   10/10    Loss: 3.5485720157623293

Epoch:   10/10    Loss: 3.597867982387543


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

I prefer starting with simple architecture, So I choose embedding_dim = 50, hidden_dim = 25, sequence_length = 30, batch_size = 50, learning_rate = 0.01 but the loss was stuck around 4.7
for many epochs even after decreasing the learning rate to 0.001.

Then I used embedding_dim = 100, hidden_dim = 50, sequence_length = 30, batch_size = 50 the loss became better around 4.3 after 10 epochs which is not good.

After that, I changed the batch_size to be 64, sequence_length = 20, embedding_dim = 200
hidden_dim = 100
loss became much better around 4.0 after 10 epochs which is not the target.

Finally I used batch_size = 64, sequence_length = 6, sequence_length = 20, embedding_dim = 500
hidden_dim = 300 for 35 epochs.

I think that I need to do more hyperparameters tuning to get better fast results but unforunately the resources are limited

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [30]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: you don't think i could have been a good time.

jerry: what are you doing here?

elaine: yeah, well i don't even know how to do it.(jerry enters)

george: what?

jerry: no, i'm gonna be a little bit in the building.

jerry: i can't.

george: what about the drake?

elaine: i don't know.

kramer: yeah.

george:(to jerry) i can't believe this!

kramer: oh! yeah!

kramer: hey.

elaine: hey, you know, i think i'm gonna be here.

kramer: well, i know, it's like a few minutes. i know you can go.(jerry leaves. the crowd is expecting me, i can't. i'm sorry. i have no idea how much they have to be able to get it out of your head.

george: oh my god, i don't know.

george: oh, i can't believe i got a lot of work to do. you know.. i know i was just thinking that i was just trying to get a new suit.

jerry:(to george) you think you should have been a little more of a man and you are not really a good fan......(turns back to kramer) hey.

george: hey.(elaine enters)

george: what do you think

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [31]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.